In [ ]:
!pip install numpy

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Simulacija vremenskih podataka u Pančevu za različite periode
def get_temperature(date_time):
    """Simulacija temperature na osnovu datuma i vremena"""
    month = date_time.month
    hour = date_time.hour

    # Temperature po mesecima
    if month in [12, 1, 2]:  # Zima
        base_temp = 5
        daily_variation = np.sin((hour - 6) * np.pi / 12) * 10  # Noću hladnije
    elif month in [3, 4, 5]:  # Proleće
        base_temp = 15
        daily_variation = np.sin((hour - 6) * np.pi / 12) * 15
    elif month in [6, 7, 8]:  # Leto
        base_temp = 30
        daily_variation = np.sin((hour - 6) * np.pi / 12) * 10
    else:  # Jesen
        base_temp = 20
        daily_variation = np.sin((hour - 6) * np.pi / 12) * 10

    # Zbir osnovne temperature i dnevnih varijacija
    return base_temp + daily_variation + random.uniform(-2, 2)  # Random noise

# Parametri dataset-a
num_sensors = 100
num_rows = 5000
sensors_temperature = np.zeros((num_rows, num_sensors))
target = np.zeros((num_rows, num_sensors))  # Binarni ciljevi (0 - bez curenja, 1 - curenje)

# Generisanje podataka
start_date = datetime(2023, 1, 1, 0, 0, 0)
for i in range(num_rows):
    date_time = start_date + timedelta(minutes=i*3)  # Svaka 3 minuta
    for j in range(num_sensors):
        temp = get_temperature(date_time)

        # Uvođenje curenja na nasumične senzore
        if random.random() < 0.02:  # 2% redova - više senzora sa curenjem
            num_leaks = random.randint(1, 3)  # Broj senzora sa curenjem
            leak_sensors = random.sample(range(num_sensors), num_leaks)
            for ls in leak_sensors:
                sensors_temperature[i, ls] = random.uniform(62, 90)  # Curenje detektovano
                target[i, ls] = 1
        else:
            if random.random() < 0.1:  # 10% redova - jedan senzor sa curenjem
                leak_sensor = random.randint(0, num_sensors - 1)
                sensors_temperature[i, leak_sensor] = random.uniform(62, 90)  # Curenje detektovano
                target[i, leak_sensor] = 1
        
        # Normalna temperatura ako nema curenja
        if target[i, j] == 0:
            sensors_temperature[i, j] = temp

# Kreiraj DataFrame
columns = [f'Sensor_{i+1}' for i in range(num_sensors)]
time_column = ['DateTime']
df = pd.DataFrame(sensors_temperature, columns=columns)
df['DateTime'] = pd.date_range(start=start_date, periods=num_rows, freq='3T')

# Target (curenje) kao poslednji deo
target_df = pd.DataFrame(target, columns=[f'Target_{i+1}' for i in range(num_sensors)])

# Kombinuj DataFrame sa ciljevima
final_df = pd.concat([df, target_df], axis=1)

# Spremi kao CSV
final_df.to_csv('simulated_sensor_data.csv', index=False)

print("Dataset uspešno generisan i sačuvan!")
